# Model5_improve

The purpose of this notebook / model experiment is to improve upon the previous fix_imbalance model achieved from model2_fix_imbalance.ipynb, by removing multicollinearity and standardizing variables (normalizing + transforming)

In [1]:
import pandas as pd
from pycaret.classification import *

## import data

In [2]:
df_ts_agg = pd.read_csv("../proData/df_ts_agg.csv")
df_ts_agg = df_ts_agg.set_index("PATIENT_ID")

df_ts_agg

ihd  Age  Gender  Height                        ICUType  \
PATIENT_ID                                                            
132539        0   54  female     NaN                   Surgical ICU   
132540        0   76    male   175.3  Cardiac Surgery Recovery Unit   
132541        0   44  female     NaN                    Medical ICU   
132543        0   68    male   180.3                    Medical ICU   
132545        0   88  female     NaN                    Medical ICU   
...         ...  ...     ...     ...                            ...   
142665        0   70  female     NaN                   Surgical ICU   
142667        0   25    male     NaN                    Medical ICU   
142670        0   44    male     NaN                    Medical ICU   
142671        1   37    male     NaN                    Medical ICU   
142673        0   78  female   157.5                   Surgical ICU   

            Weight_min  Weight_max  Weight_med  GCS_min  GCS_max  ...  \
PATIENT_ID                                                        ...   
132539             NaN         NaN         NaN     14.0     15.0  ...   
132540            76.0        81.6        80.6      3.0     15.0  ...   
132541            56.7        56.7        56.7      5.0      8.0  ...   
132543            84.6        84.6        84.6     14.0     15.0  ...   
132545             NaN         NaN         NaN     15.0     15.0  ...   
...                ...         ...         ...      ...      ...  ...   
142665            87.0        87.0        87.0      3.0     15.0  ...   
142667           166.4       166.4       166.4     15.0     15.0  ...   
142670           109.0       109.0       109.0      3.0      8.0  ...   
142671            87.4        87.4        87.4      3.0      7.0  ...   
142673            70.7        88.8        88.8      3.0     15.0  ...   

            Lactate_med  Cholesterol_min  Cholesterol_max  Cholesterol_med  \
PATIENT_ID                                                                   
132539              NaN              NaN              NaN              NaN   
132540              NaN              NaN              NaN              NaN   
132541              1.3              NaN              NaN              NaN   
132543              NaN              NaN              NaN              NaN   
132545              NaN              NaN              NaN              NaN   
...                 ...              ...              ...              ...   
142665              2.3              NaN              NaN              NaN   
142667              NaN            117.0            117.0            117.0   
142670              NaN              NaN              NaN              NaN   
142671              1.9              NaN              NaN              NaN   
142673              2.6              NaN              NaN              NaN   

            TroponinI_min  TroponinI_max  TroponinI_med  TroponinT_min  \
PATIENT_ID                                                               
132539                NaN            NaN            NaN            NaN   
132540                NaN            NaN            NaN            NaN   
132541                NaN            NaN            NaN            NaN   
132543                NaN            NaN            NaN            NaN   
132545                NaN            NaN            NaN            NaN   
...                   ...            ...            ...            ...   
142665                NaN            NaN            NaN            NaN   
142667                NaN            NaN            NaN            NaN   
142670                NaN            NaN            NaN            NaN   
142671                NaN            NaN            NaN            NaN   
142673                NaN            NaN            NaN            NaN   

            TroponinT_max  TroponinT_med  
PATIENT_ID                                
132539                NaN            NaN  
132540                NaN      

## setup experiment with pycaret

enable all previously tested improvement parameters
* fix_imbalance
* remove_multicollinearity
* normalize and transformation

In [3]:
exp_physionet = setup(data = df_ts_agg, target = 'ihd', session_id=123,
                      numeric_features = ["GCS_min", "GCS_max"],
                      normalize = True, 
                      transformation = True, 
                      #ignore_low_variance = True,
                      remove_multicollinearity = True, multicollinearity_threshold = 0.9,
                      fix_imbalance = True
                     ) 

Setup Succesfully Completed!


## compare common classification models

min(Recall, Precision) is used for comparing, as suggested by the original physionet 2012 challenge

In [41]:
compare_models(turbo = False)

## create, tune, evaluate, predict adaboost model

In [4]:
ada = create_model("ada")

In [10]:
tuned_ada = tune_model(ada, optimize = "Accuracy", choose_better = True)

In [11]:
evaluate_model(tuned_ada)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [12]:
predict_model(tuned_ada)

Model  Accuracy     AUC  Recall   Prec.      F1   Kappa  \
0  Ada Boost Classifier    0.8376  0.8097  0.3976  0.4099  0.4037  0.3097   

      MCC  
0  0.3098

Age    Height  Weight_max   GCS_min   GCS_max   GCS_med    HR_min  \
0     0.477499 -1.215611    0.925603 -1.152209  0.672376 -0.511354  0.753785   
1    -0.556735 -0.031642   -0.694846  1.181913  0.672376  0.963869  1.427515   
2     1.769352  0.035305   -0.633653  0.669598  0.672376  0.434389 -1.360105   
3    -1.189793 -0.031642    0.178626  1.341102  0.672376  0.963869  0.616506   
4    -0.057864 -0.031642   -2.314832  0.288248 -1.528718 -0.862434 -1.360105   
...        ...       ...         ...       ...       ...       ...       ...   
1196  1.009600 -1.509279   -0.725754  0.483924 -0.551305 -0.364897  0.055875   
1197 -0.505915 -0.031642   -0.478512  1.181913  0.672376  0.963869 -0.827196   
1198  0.406102 -0.031642   -0.819713  1.181913  0.672376  0.963869  2.408744   
1199 -2.067770 -0.031642    0.439906 -0.142452 -0.551305 -1.206565  1.025611   
1200  1.089991 -2.097840   -0.329244 -1.152209  0.672376 -0.640920  0.822069   

        HR_max    HR_med  NIDiasABP_min  ...  \
0    -0.510431  0.160269      -0.915253  ...   
1     1.920192  1.464024      -0.820052  ...   
2     0.149794 -0.498416      -0.004823  ...   
3     0.103027  0.517847       0.721668  ...   
4    -1.641198 -1.738028       0.721668  ...   
...        ...       ...            ...  ...   
1196 -0.295979 -0.320230      -0.447804  ...   
1197  0.704247 -0.606748      -0.004823  ...   
1198  1.190597  2.268197      -0.632149  ...   
1199  0.501095  1.106328       1.170347  ...   
1200 -0.906773 -0.320230       0.064562  ...   

      ICUType_Cardiac Surgery Recovery Unit  ICUType_Coronary Care Unit  \
0                                       1.0                         0.0   
1                                       0.0                         0.0   
2                                       0.0                         0.0   
3                                       0.0                         0.0   
4                                       1.0                         0.0   
...                                     ...                         ...   
1196                                    0.0                         0.0   
1197                                    1.0                         0.0   
1198                                    0.0                         0.0   
1199                                    0.0                         0.0   
1200                                    1.0                         0.0   

      ICUType_Medical ICU  ICUType_Surgical ICU  MechVent_min_0  \
0                     0.0                   0.0             1.0   
1                     1.0                   0.0             1.0   
2                     0.0                   1.0             1.0   
3                     0.0                   1.0             1.0   
4                     0.0                   0.0             1.0   
...                   ...                   ...             ...   
1196                  0.0                   1.0             1.0   
1197                  0.0                   0.0             1.0   
1198                  1.0                   0.0             1.0   
1199                  0.0                   1.0             1.0   
1200                  0.0                   0.0             1.0   

      MechVent_max_1  MechVent_med_0.5  ihd  Label   Score  
0                1.0               0.0    0      0  0.4866  
1                0.0               0.0    0      0  0.4939  
2                0.0               0.0    1      1  0.5040  
3                0.0               0.0    0      0  0.4805  
4                1.0               0.0    0      0  0.4925  
...              ...               ...  ...    ...     ...  
1196             0.0               0.0    1      1  0.5001  
1197             0.0               0.0    0      0  0.4813  
1198             0.0               0.0    1      0  0.4988  
1199             1.0               0.0    0      0  0.4960  
1200             1.0               0.0    0      0  0.4930  

[1201 rows x 91 columns]

## explore blend models, both hard and soft voting

In [13]:
blend_hard = blend_models()

In [19]:
evaluate_model(blend_hard)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [15]:
predict_model(blend_hard)

Model  Accuracy  AUC  Recall   Prec.     F1   Kappa    MCC
0  Voting Classifier    0.8293    0  0.4398  0.3946  0.416  0.3163  0.317

Age    Height  Weight_max   GCS_min   GCS_max   GCS_med    HR_min  \
0     0.477499 -1.215611    0.925603 -1.152209  0.672376 -0.511354  0.753785   
1    -0.556735 -0.031642   -0.694846  1.181913  0.672376  0.963869  1.427515   
2     1.769352  0.035305   -0.633653  0.669598  0.672376  0.434389 -1.360105   
3    -1.189793 -0.031642    0.178626  1.341102  0.672376  0.963869  0.616506   
4    -0.057864 -0.031642   -2.314832  0.288248 -1.528718 -0.862434 -1.360105   
...        ...       ...         ...       ...       ...       ...       ...   
1196  1.009600 -1.509279   -0.725754  0.483924 -0.551305 -0.364897  0.055875   
1197 -0.505915 -0.031642   -0.478512  1.181913  0.672376  0.963869 -0.827196   
1198  0.406102 -0.031642   -0.819713  1.181913  0.672376  0.963869  2.408744   
1199 -2.067770 -0.031642    0.439906 -0.142452 -0.551305 -1.206565  1.025611   
1200  1.089991 -2.097840   -0.329244 -1.152209  0.672376 -0.640920  0.822069   

        HR_max    HR_med  NIDiasABP_min  ...  Gender_not_available  \
0    -0.510431  0.160269      -0.915253  ...                   0.0   
1     1.920192  1.464024      -0.820052  ...                   0.0   
2     0.149794 -0.498416      -0.004823  ...                   0.0   
3     0.103027  0.517847       0.721668  ...                   0.0   
4    -1.641198 -1.738028       0.721668  ...                   0.0   
...        ...       ...            ...  ...                   ...   
1196 -0.295979 -0.320230      -0.447804  ...                   0.0   
1197  0.704247 -0.606748      -0.004823  ...                   0.0   
1198  1.190597  2.268197      -0.632149  ...                   0.0   
1199  0.501095  1.106328       1.170347  ...                   0.0   
1200 -0.906773 -0.320230       0.064562  ...                   0.0   

      ICUType_Cardiac Surgery Recovery Unit  ICUType_Coronary Care Unit  \
0                                       1.0                         0.0   
1                                       0.0                         0.0   
2                                       0.0                         0.0   
3                                       0.0                         0.0   
4                                       1.0                         0.0   
...                                     ...                         ...   
1196                                    0.0                         0.0   
1197                                    1.0                         0.0   
1198                                    0.0                         0.0   
1199                                    0.0                         0.0   
1200                                    1.0                         0.0   

      ICUType_Medical ICU  ICUType_Surgical ICU  MechVent_min_0  \
0                     0.0                   0.0             1.0   
1                     1.0                   0.0             1.0   
2                     0.0                   1.0             1.0   
3                     0.0                   1.0             1.0   
4                     0.0                   0.0             1.0   
...                   ...                   ...             ...   
1196                  0.0                   1.0             1.0   
1197                  0.0                   0.0             1.0   
1198                  1.0                   0.0             1.0   
1199                  0.0                   1.0             1.0   
1200                  0.0                   0.0             1.0   

      MechVent_max_1  MechVent_med_0.5  ihd  Label  
0                1.0               0.0    0      0  
1                0.0               0.0    0      0  
2                0.0               0.0    1      0  
3                0.0               0.0    0      0  
4                1.0               0.0    0      0  
...              ...               ...  ...    ...  
1196             0.0               0.0    1      0  
1197             0.0               0.0    0      0  
1198             0.0      

In [16]:
blend_soft = blend_models(method = "soft")

In [17]:
evaluate_model(blend_soft)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [18]:
predict_model(blend_soft)

Model  Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
0  Voting Classifier    0.8318  0.8098  0.4699  0.4062  0.4358  0.3375  0.3388

Age    Height  Weight_max   GCS_min   GCS_max   GCS_med    HR_min  \
0     0.477499 -1.215611    0.925603 -1.152209  0.672376 -0.511354  0.753785   
1    -0.556735 -0.031642   -0.694846  1.181913  0.672376  0.963869  1.427515   
2     1.769352  0.035305   -0.633653  0.669598  0.672376  0.434389 -1.360105   
3    -1.189793 -0.031642    0.178626  1.341102  0.672376  0.963869  0.616506   
4    -0.057864 -0.031642   -2.314832  0.288248 -1.528718 -0.862434 -1.360105   
...        ...       ...         ...       ...       ...       ...       ...   
1196  1.009600 -1.509279   -0.725754  0.483924 -0.551305 -0.364897  0.055875   
1197 -0.505915 -0.031642   -0.478512  1.181913  0.672376  0.963869 -0.827196   
1198  0.406102 -0.031642   -0.819713  1.181913  0.672376  0.963869  2.408744   
1199 -2.067770 -0.031642    0.439906 -0.142452 -0.551305 -1.206565  1.025611   
1200  1.089991 -2.097840   -0.329244 -1.152209  0.672376 -0.640920  0.822069   

        HR_max    HR_med  NIDiasABP_min  ...  \
0    -0.510431  0.160269      -0.915253  ...   
1     1.920192  1.464024      -0.820052  ...   
2     0.149794 -0.498416      -0.004823  ...   
3     0.103027  0.517847       0.721668  ...   
4    -1.641198 -1.738028       0.721668  ...   
...        ...       ...            ...  ...   
1196 -0.295979 -0.320230      -0.447804  ...   
1197  0.704247 -0.606748      -0.004823  ...   
1198  1.190597  2.268197      -0.632149  ...   
1199  0.501095  1.106328       1.170347  ...   
1200 -0.906773 -0.320230       0.064562  ...   

      ICUType_Cardiac Surgery Recovery Unit  ICUType_Coronary Care Unit  \
0                                       1.0                         0.0   
1                                       0.0                         0.0   
2                                       0.0                         0.0   
3                                       0.0                         0.0   
4                                       1.0                         0.0   
...                                     ...                         ...   
1196                                    0.0                         0.0   
1197                                    1.0                         0.0   
1198                                    0.0                         0.0   
1199                                    0.0                         0.0   
1200                                    1.0                         0.0   

      ICUType_Medical ICU  ICUType_Surgical ICU  MechVent_min_0  \
0                     0.0                   0.0             1.0   
1                     1.0                   0.0             1.0   
2                     0.0                   1.0             1.0   
3                     0.0                   1.0             1.0   
4                     0.0                   0.0             1.0   
...                   ...                   ...             ...   
1196                  0.0                   1.0             1.0   
1197                  0.0                   0.0             1.0   
1198                  1.0                   0.0             1.0   
1199                  0.0                   1.0             1.0   
1200                  0.0                   0.0             1.0   

      MechVent_max_1  MechVent_med_0.5  ihd  Label   Score  
0                1.0               0.0    0      0  0.1323  
1                0.0               0.0    0      0  0.3641  
2                0.0               0.0    1      0  0.4384  
3                0.0               0.0    0      0  0.0470  
4                1.0               0.0    0      0  0.3850  
...              ...               ...  ...    ...     ...  
1196             0.0               0.0    1      1  0.5178  
1197             0.0               0.0    0      0  0.1164  
1198             0.0               0.0    1      0  0.3089  
1199             1.0               0.0    0      0  0.2914  
1200             1.0               0.0    0      0  0.3020  

[1201 rows x 91 columns]

## conclusion

combining all: fixing imbalanced label, multicollinearity removal, normalization / transformation, either a single model, e.g. adaboost, or blend models has increased train or test min(recall, precision) to 0.41